# Faster R-CNN Implementation

##Toolbox

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from keras import backend as K
from keras.optimizers import Adam, SGD, RMSprop
from keras.layers import Flatten, Dense, Input, Conv2D, MaxPooling2D, Dropout, Layer, Concatenate
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, TimeDistributed

##Architecture

##Input

In [0]:
img_height = 224
img_width = 224
img_depth = 3
input_shape = (img_height,img_width,img_depth)

def input_tensor(input_shape):
  x = Input(shape = input_shape)
  return x

## Base Model

In [0]:
vgg_ratio = 16

In [0]:
def vgg16(input_tensor):
      
    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(input_tensor)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)

    return x

In [0]:
def get_weights(weights):
  return None

##RPN

In [0]:
rpn_height = img_height/vgg_ratio
rpn_width = img_width/vgg_ratio
rpn_depth = 512
anchors_ratios = [0.5,1,1.5]
anchors_size = [64,128,256]
anchors_count = len(anchors_ratios) * len(anchors_size)


def rpn(x):
  
  # Mutual Layer
  x1 = Conv2D(512,(3,3), activation='relu', padding='same', kernel_initializer='normal', name='rpn_mutual_layer')(x)
  
  # Classification Layer
  x_class = Conv2D(anchors_count, (1,1), activation = 'sigmoid', kernel_initializer = 'uniform', name='rpn_class_layer')(x)
  
  # Regression Layer
  x_reg = Conv2D(4*anchors_count, (1,1), activation='linear', kernel_initializer='zero', name='rpn_reg_layer')(x)
  return [x_class,x_reg]

##ROI

In [0]:
class ROIPoolingLayer(Layer):
    '''
    Input will be : [VGG16 Feature Layers, Proposal]
    Shape is [(1,rpn_width,rpn_height,512),(1,x,y,h,w)]
    '''
    def __init__(self, output_dim,pooling_size, **kwargs):
        
        self.pooling_size = pooling_size
        
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.nb_channels = input_shape[0][3]

    def call(self, x):
        assert len(x) == 2
        img = x[0]
        roi = x[1]
        
        x = roi[0,0]
        y = roi[0,1]
        h = roi[0,2]
        w = roi[0,3]
        
        x = K.cast(x, 'int32')
        y = K.cast(y, 'int32')
        w = K.cast(w, 'int32')
        h = K.cast(h, 'int32')
        
        output = tf.image.resize((img[:, y:y+h, x:x+w, :], (self.pool_size, self.pool_size)))
        
        output = K.reshape(output , (1, self.pool_size, self.pool_size, self.nb_channels))
        
        return output

    def compute_output_shape(self, input_shape):
        return None, self.pool_size, self.pool_size, self.nb_channels

In [0]:
input_test = input_tensor(input_shape)
vgg16_model = vgg16(input_test)
output = rpn(vgg16_model)
print(output)

##R-CNN